In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/CS 6158/CS6158 Research Project/src/6158_final_project'

/content/gdrive/MyDrive/CS 6158/CS6158 Research Project/src/6158_final_project


In [3]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    pipeline)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd
import json
import requests
from sklearn.model_selection import train_test_split
import os

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("./nofuneval_memory_model")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Some weights of the model checkpoint at ./nofuneval_memory_model were not used when initializing LlamaForCausalLM: ['model.layers.0.self_attn.q_proj.base_layer.weight', 'model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight', 'model.layers.1.self_attn.q_proj.lora_A.default.weight', 'model.layers.1.self_attn.q_proj.lora_B.default.weight', 'model.layers.1.self_attn.v_proj.base_layer.weight', 'model.layers.1.self_attn.v_proj.lora_A.default.weight', 'model.layers.1.self_attn.v_proj.lora_B.default.weight', 'model.layers.10.self_attn.q_proj.base_layer.weight', 'model.layers.10.self_attn.q_proj.lora_A.default.weight', 'model.layers.10.self_attn.q_proj.lora_B.default.weight', 'model.layers.10.self_attn.v_proj.base_layer.w

In [6]:
print(os.getcwd())

/content/gdrive/MyDrive/CS 6158/CS6158 Research Project/src/6158_final_project


In [7]:
%cd ../datasets

/content/gdrive/MyDrive/CS 6158/CS6158 Research Project/src/datasets


In [8]:
!ls

test  test.json  train	train.json


In [9]:
# load Dataset from folder saved by Dataset library
test_dataset = Dataset.load_from_disk("./test")

In [10]:
print(test_dataset)

Dataset({
    features: ['non_functional_requirement', 'commit', 'commit_message', 'source_code', 'target_code', 'pl', 'chain_of_thought', 'one_shot', 'base_prompt', 'coding_concepts', 'classification_left_prompt', 'classification_left_label', 'classification_right_prompt', 'classification_right_label', 'input_text', 'output_text', '__index_level_0__'],
    num_rows: 11
})


In [11]:
def tokenize_function(examples):
  return tokenizer(examples['input_text'], examples['output_text'], max_length=2048, padding="max_length", truncation=True)

tokenized_dataset = test_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.map(
  lambda examples: {'labels': examples['input_ids']},
  batched=True
)

In [12]:
print(tokenized_dataset)

Dataset({
    features: ['non_functional_requirement', 'commit', 'commit_message', 'source_code', 'target_code', 'pl', 'chain_of_thought', 'one_shot', 'base_prompt', 'coding_concepts', 'classification_left_prompt', 'classification_left_label', 'classification_right_prompt', 'classification_right_label', 'input_text', 'output_text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11
})


In [14]:
inputs = torch.tensor(tokenized_dataset['input_ids'])
inputs = inputs.to(device='cuda')
model = model.to(device='cuda')
# print(inputs.device)
generations = []
for input in inputs:
  input = input.unsqueeze(0)
  output = model.generate(input_ids=input, max_length=2048, max_new_tokens=2048, num_beams=5)
  generation = tokenizer.decode(output[0], skip_special_tokens=True)
  print(generation)
  generations.append(generation)
# output = model.generate(input_ids=inputs, max_length=2048, max_new_tokens=2048, num_beams=5)
# generation = tokenizer.decode(output[0], skip_special_tokens=True)
# print(generation)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generati

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given scala program to optimize and improve the energy usage. Write the entire code and no other text in the response.
```scala
package com.github.plokhotnyuk.actors

import java.util.concurrent._
import java.util.concurrent.atomic.{AtomicReference, AtomicInteger}
import java.lang.InterruptedException
import scala.annotation.tailrec
import java.util.concurrent.locks.LockSupport

/**
 * A high performance implementation of thread pool with fixed number of threads.
 *
 * Implementation of task queue based on non-intrusive MPSC node-based queue, described by Dmitriy Vyukov:
 * http://www.1024cores.net/home/lock-free-algorithms/queues/non-intrusive-mpsc-node-based-queue
 *
 * @param threadCount a number of worker threads in pool
 * @param threadFactory a factory to be used to build worker threads
 * @param handler the handler for internal worker threads th

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given javascript program to optimize and improve the energy usage. Write the entire code and no other text in the response.
```javascript
/* -*- Mode: Java; tab-width: 2; indent-tabs-mode: nil; c-basic-offset: 2 -*- /
/* vim: set shiftwidth=2 tabstop=2 autoindent cindent expandtab: */

'use strict';

var BatteryManager = {
  TOASTER_TIMEOUT: 5000,
  TRANSITION_SPEED: 1.8,
  TRANSITION_FRACTION: 0.30,

  AUTO_SHUTDOWN_LEVEL: 0.02,

  _notification: null,
  _screenOn: true,
  _previousLevel: 0,

  getAllElements: function bm_getAllElements() {
    this.screen = document.getElementById('screen');
    this.overlay = document.getElementById('system-overlay');
    this.notification = document.getElementById('battery');
  },

  checkBatteryDrainage: function bm_checkBatteryDrainage() {
    var battery = window.navigator.battery;
    if (!battery)
      return

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given java program to optimize and improve the memory usage. Write the entire code and no other text in the response.
```java
package org.fdroid.fdroid;

import android.app.Instrumentation;
import android.os.Build;
import android.support.test.InstrumentationRegistry;
import android.support.test.espresso.IdlingPolicies;
import android.support.test.espresso.ViewInteraction;
import android.support.test.filters.LargeTest;
import android.support.test.rule.ActivityTestRule;
import android.support.test.runner.AndroidJUnit4;
import android.support.test.uiautomator.UiDevice;
import android.support.test.uiautomator.UiObject;
import android.support.test.uiautomator.UiObjectNotFoundException;
import android.support.test.uiautomator.UiSelector;
import android.util.Log;
import android.view.View;
import org.fdroid.fdroid.views.BannerUpdatingRepos;
import org.fdroid.f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given java program to optimize and improve the memory usage. Write the entire code and no other text in the response.
```java
package com.aaronjwood.portauthority.async;

import android.os.AsyncTask;

import com.aaronjwood.portauthority.response.MainAsyncResponse;

import java.io.IOException;
import java.lang.ref.WeakReference;

import okhttp3.OkHttpClient;
import okhttp3.Request;
import okhttp3.Response;

public class WanIpAsyncTask extends AsyncTask<Void, Void, String> {

    // IP service is 100% open source https://github.com/aaronjwood/public-ip-api
    private static final String EXTERNAL_IP_SERVICE = "https://public-ip-api.appspot.com/";
    private final WeakReference<MainAsyncResponse> delegate;

    /**
     * Constructor to set the delegate
     *
     * @param delegate Called when the external IP has been fetched
     */
    public WanIpAsy

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given java program to optimize and improve the memory usage. Write the entire code and no other text in the response.
```java
/* Copyright (C) 2016 Julian Andres Klode <jak@jak-linux.org>
 *
 * Derived from AdBuster:
 * Copyright (C) 2016 Daniel Brodie <dbrodie@gmail.com>
 *
 * This program is free software: you can redistribute it and/or modify
 * it under the terms of the GNU General Public License as published by
 * the Free Software Foundation, version 3.
 *
 * Contributions shall also be provided under any later versions of the
 * GPL.
 */
package org.jak_linux.dns66.vpn;

import android.app.Notification;
import android.app.PendingIntent;
import android.app.Service;
import android.content.BroadcastReceiver;
import android.content.Context;
import android.content.Intent;
import android.content.IntentFilter;
import android.net.ConnectivityManager;
im

In [20]:
print(generations[2])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given objectivec program to optimize and improve the energy usage. Write the entire code and no other text in the response.
```objectivec
#import "MIDIBridgeAppDelegate.h"
#import "MIDIClient.h"
#import "IPCRouter.h"
#import "LogWindowController.h"

#pragma mark Private members

@interface MIDIBridgeAppDelegate ()
{
    MIDIClient *_midiClient;
    IPCRouter *_ipcRouter;
    LogWindowController *_logWindowController;
    
    NSStatusItem *_statusItem;
    NSInteger _signalCount;
}
@end

#pragma mark
#pragma mark Application delegate class implementation

@implementation MIDIBridgeAppDelegate

- (void)applicationDidFinishLaunching:(NSNotification *)aNotification
{
    _midiClient = [[MIDIClient alloc] initWithDelegate:self];
    _ipcRouter = [[IPCRouter alloc] initWithDelegate:self];
    _logWindowController = [[LogWindowController alloc] initWithWindo

In [19]:
print(test_dataset[2]['input_text'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the given objectivec program to optimize and improve the energy usage. Write the entire code and no other text in the response.
```objectivec
#import "MIDIBridgeAppDelegate.h"
#import "MIDIClient.h"
#import "IPCRouter.h"
#import "LogWindowController.h"

#pragma mark Private members

@interface MIDIBridgeAppDelegate ()
{
    MIDIClient *_midiClient;
    IPCRouter *_ipcRouter;
    LogWindowController *_logWindowController;
    
    NSStatusItem *_statusItem;
    NSInteger _signalCount;
}
@end

#pragma mark
#pragma mark Application delegate class implementation

@implementation MIDIBridgeAppDelegate

- (void)applicationDidFinishLaunching:(NSNotification *)aNotification
{
    _midiClient = [[MIDIClient alloc] initWithDelegate:self];
    _ipcRouter = [[IPCRouter alloc] initWithDelegate:self];
    _logWindowController = [[LogWindowController alloc] initWithWindo

In [21]:
# Save the generations elsewhere
with open('generations.json', 'w') as f:
    json.dump(generations, f)

In [ ]:
# Switch the model out for the default meta 3.2 1B model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
model = model.to(device='cuda')

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

In [ ]:
generations_base = []
for input in inputs:
  input = input.unsqueeze(0)
  output = model.generate(input_ids=input, max_length=2048, max_new_tokens=2048, num_beams=5)
  generation = tokenizer.decode(output[0], skip_special_tokens=True)
  print(generation)
  generations_base.append(generation)

In [ ]:
with open('generations_base.json', 'w') as f:
    json.dump(generations, f)